In [1]:
from timeit import Timer
t = Timer("filepath = '/content/drive/MyDrive/WordEmbedding/e5-small-v2.zip'; \
         extract_to = '/'; \
         from zipfile import ZipFile; \
         ZipFile(filepath).extractall(path=extract_to)")
t.timeit(1)

3.269984722000004

In [ ]:
pip install datasets

In [3]:
from datasets import load_dataset  # !pip install datasets

squad_dev = load_dataset('squad_v2', split='validation')



Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [4]:
squad_dev[2]

{'id': '56ddde6b9a695914005b962a',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'From which countries did the Norse originate?',
 'answers': {'text': ['Denmark, Iceland and Norway',
   'Denmark, Iceland and Norway',
   'Denmark, Iceland and Norway',
   

In [5]:
contexts = []
contexts=squad_dev['context']
# loop through the context passages
# for record in squad_dev['context']:
#    # join context passages for each question and append to contexts list
#    contexts.append('\n'.join(record['contexts']))
# view some of the contexts
# for context in contexts[:2]:
#    print(f"{context[:300]}...")


In [ ]:
pip install transformers

In [7]:
from transformers import BertTokenizerFast  # !pip install transformers

# load bert tokenizer from huggingface
tokenizer = BertTokenizerFast.from_pretrained(
   'bert-base-uncased'
)


(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
# tokenize the context passage
inputs = tokenizer(
   contexts[0], padding=True, truncation=True,
   max_length=512
)
inputs.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
input_ids = inputs['input_ids']
input_ids


In [ ]:
from collections import Counter

# convert the input_ids list to a dictionary of key to frequency values
sparse_vec = dict(Counter(input_ids))
sparse_vec


In [11]:
def build_dict(input_batch):
 # store a batch of sparse embeddings
   sparse_emb = []
   # iterate through input batch
   for token_ids in input_batch:
       indices = []
       values = []
       # convert the input_ids list to a dictionary of key to frequency values
       d = dict(Counter(token_ids))
       for idx in d:
          indices.append(idx)
          values.append(float(d[idx]))  # Cast the frequency count to a float
       sparse_emb.append({'indices': indices, 'values': values})

   # return sparse_emb list
   return sparse_emb


def generate_sparse_vectors(context_batch):
  # create batch of input_ids
  inputs = tokenizer(
          context_batch, padding=True,
          truncation=True,
          max_length=512
  )['input_ids']
  # create sparse dictionaries
  sparse_embeds = build_dict(inputs)
  return sparse_embeds


In [ ]:
pip install sentence_transformers

Dense vectors

In [13]:
# !pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('/content/e5-small-v2')

emb = model.encode(contexts[0])
emb.shape


(384,)

In [ ]:
pip install pinecone-client

In [15]:
import pinecone  # !pip install pinecone-client

pinecone.init(
   api_key="898a7037-44eb-49bb-94a0-95219be317da",  # app.pinecone.io
   environment="gcp-starter"  # find next to api key in console
)
# choose a name for your index
index_name = "hybrid-search-intro"

# create the index
pinecone.create_index(
   name = index_name,
   dimension = 384,  # dimensionality of dense model
   metric = "dotproduct",
   pod_type = "s1"
)


In [16]:
index = pinecone.Index('hybrid-search-intro')

In [17]:
from tqdm.auto import tqdm

batch_size = 32

for i in tqdm(range(0, len(contexts), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(contexts))
    # extract batch
    context_batch = contexts[i:i_end]
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]
    # add context passages as metadata
    meta = [{'context': context} for context in context_batch]
    # create dense vectors
    dense_embeds = model.encode(context_batch).tolist()
    # create sparse vectors
    sparse_embeds = generate_sparse_vectors(context_batch)

    vectors = []
    # loop through the data and create dictionaries for upserts
    for _id, sparse, dense, metadata in zip(
        ids, sparse_embeds, dense_embeds, meta
    ):
      sparse_values_float = [float(value) for value in sparse['values']]

      vectors.append({
          'id': _id,
          'sparse_values': {
              'indices': sparse['indices'],
              'values': sparse_values_float  # Using the converted float values
          },
          'values': dense,
          'metadata': metadata
      })

    # upload the documents to the new hybrid index
    index.upsert(vectors)



  0%|          | 0/372 [00:00<?, ?it/s]

In [18]:
# show index description after uploading the documents
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.11488,
 'namespaces': {'': {'vector_count': 11488}},
 'total_vector_count': 11488}

In [24]:
def hybrid_scale(dense, sparse, alpha: float):
    # check alpha value is in range
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    # scale sparse and dense vectors to create hybrid search vecs
    hsparse = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    hdense = [v * alpha for v in dense]
    return hdense, hsparse


def hybrid_query(question, top_k, alpha):
   # convert the question into a sparse vector
   sparse_vec = generate_sparse_vectors([question])[0]
   # convert the question into a dense vector
   dense_vec = model.encode([question]).tolist()
   # scale alpha with hybrid_scale
   dense_vec, sparse_vec = hybrid_scale(
      dense_vec, sparse_vec, alpha
   )
   # query pinecone with the query parameters
   result = index.query(
      vector=dense_vec,
      sparse_vector=sparse_vec,
      top_k=top_k,include_metadata=True
   )
   # return search results as json
   return result


In [20]:
question = "what is American Baptist Education?"


In [25]:
hybrid_query(question, top_k=2,  alpha=1)

{'matches': [{'id': '6039',
              'metadata': {'context': 'Religiously affiliated and '
                                      'denominational schools form a '
                                      'subcategory of private schools. Some '
                                      'such schools teach religious education, '
                                      'together with the usual academic '
                                      'subjects to impress their particular '
                                      "faith's beliefs and traditions in the "
                                      'students who attend. Others use the '
                                      'denomination as more of a general label '
                                      'to describe on what the founders based '
                                      'their belief, while still maintaining a '
                                      'fine distinction between academics and '
                                      'relig